#**Caso 2. Ejemplo de creación de un modelo con Random Forest**

# **Instalación de librerías**

**Instalación de librerías**
Paso 1. Revisar la versión de R instadala

In [ ]:
R.version.string

In [ ]:
install.packages("caret")
install.packages("randomForest")
install.packages("dplyr")
install.packages("Boruta")

Cargando las librerías

In [ ]:
library(caret)
library(randomForest)
library(dplyr)
library(Boruta)

# Load Data From CSV
Este ejemplo muestra como cargar un archivo csv de un directorio local.

Primero verificamos en que directorio nos encontramos:

In [ ]:
#getwd()

Implementamos una semilla para que los datos sean siempre los mismos valores en nuestras pruebas

In [ ]:
set.seed(7)

In [ ]:

#datos <- read.csv(file = 'datav3.csv', sep = ',')
inputData <- read.csv('sample_data/datav3.csv', sep = ',') 
head(inputData)

Revisamos el contenido del dataset cargado

In [ ]:
glimpse(inputData)

Cambiando de tipo char a factor la clase que queremos clasificar.

In [ ]:
inputData$result = as.factor(inputData$result)

In [ ]:
#Revisando la clase
table(inputData$result)

# Buscando los mejores atributos con Boruta


In [ ]:
set.seed(123)

Boruta es un algoritmo envolvente de selección de características relevantes, capaz de trabajar con cualquier método de clasificación; por defecto, Boruta utiliza Random Forest. 
El método realiza una búsqueda descendente de características relevantes comparando la importancia de los atributos originales con la importancia alcanzable al azar, estimada mediante sus copias permutadas, y eliminando progresivamente las características irrelevantes para estabilizar esa prueba.


In [ ]:
# Decidir si una variable es importante o no utilizando Boruta
boruta_output <- Boruta(result ~ ., data=na.omit(inputData), doTrace=2, maxRuns=500)  # perform Boruta search
names(boruta_output)

In [ ]:
# Obtener variables significativas, incluyendo las tentativas
boruta_signif <- getSelectedAttributes(boruta_output, withTentative = TRUE)
print(boruta_signif)  

In [ ]:
# Hacer un arreglo tentativo
roughFixMod <- TentativeRoughFix(boruta_output)
print(roughFixMod)
attStats(boruta_output)
boruta_signif <- getSelectedAttributes(roughFixMod)
print(boruta_signif)


In [ ]:
# Puntuación de la importancia de las variables
imps <- attStats(roughFixMod)
imps2 = imps[imps$decision != 'Rejected', c('meanImp', 'decision')]
head(imps2[order(-imps2$meanImp), ])  # descending sort

In [ ]:
# Graficar la importancia de las variables
plot(boruta_output, cex.axis=.7, las=2, xlab="", main="Variable Importance")  

# Particionando datos para el modelo

In [ ]:
set.seed(123)

In [ ]:
# Se crean los índices de las observaciones de entrenamiento
train <- createDataPartition(y = inputData$result, p = 0.7, list = FALSE, times = 1)
datos_train <- inputData[train, ]
datos_test  <- inputData[-train, ]

In [ ]:
# Resumen del set de datos train
glimpse(datos_train)

# Creando RandomForest para un modelo de predicción de Taquicardia


In [ ]:
#tune mtry
t <- tuneRF(datos_train[,-25], datos_train[,25],
            stepFactor = 0.5,
            plot = TRUE,
            ntreeTry = 400,
            trace =  TRUE,
            improve = 0.05)

In [ ]:
rfT <- randomForest(result ~ edad + calcio_en_sangre + taquicardia_sinusal_ecg,
                    data=datos_train,
                    ntree = 400, #Número de árboles a crear. No debe establecerse un número demasiado pequeño, para asegurar que cada fila de entrada se predice al menos un par de veces.
                    mtry = 2, #Número de variables muestreadas aleatoriamente como candidatas en cada división.
                    importance = TRUE, #¿Debe evaluarse la importancia de los predictores?
                    proximity = TRUE) #¿Se debe calcular la medida de proximidad entre las filas?

In [ ]:
rfT

In [ ]:
# Importancia de las variables
rfT$importance

In [ ]:
plot(rfT)

# Realizando la predicción de datos

In [ ]:
p <- predict(rfT, datos_test)

In [ ]:
confusionMatrix(p, datos_test$result)

In [ ]:
# Compute model accuracy rate
porcentaje <- mean(p == datos_test$result)*100
paste("Porcentaje de predicción RF:",round(porcentaje,2))

In [ ]:
# Error de test
error_test <- mean(p != datos_test$result)
paste("El error de test del modelo RF:", round(error_test*100, 2), "%")


In [ ]:
#Salvando el modelo
saveRDS(rfT,"rfT.rds")

# Lanzar en una API

Ejemplo de definición de API

Mostrar el modelo de taquicardia trabajando en:

http://162.214.187.236:8002/__docs__/